In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 75%; }
</style>
"""))

In [2]:
!dir

script					 training_ground-handy.ipynb
training_ground-handy-ca1_ca2_ca3.ipynb  training_ground.ipynb


In [3]:
from platform import python_version

print(python_version())

3.8.3


In [4]:
%matplotlib inline
import os
import sys
import gc
import time
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model

module_path_list = [os.path.abspath(os.path.join('../')), 
                    os.path.abspath(os.path.join('../../RCSnail-Commons'))]

for module_path in module_path_list:
    if module_path not in sys.path:
        sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
#from src.utilities.transformer import Transformer
from src.learning.training.generator import Generator, GenFiles
#from src.learning.models import create_standalone_nvidia_cnn, create_standalone_resnet
                                

In [5]:
def create_standalone_nvidia_cnn(activation='linear', input_shape=(60, 180, 3), output_shape=1):
    """
    Activation: linear, softmax.
    Architecture is from nvidia paper mentioned in https://github.com/tanelp/self-driving-convnet/blob/master/train.py
    """
    from tensorflow.keras.layers import Convolution2D
    from tensorflow.keras.regularizers import l2
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.layers import Flatten
    from tensorflow.keras.layers import Input
    from tensorflow.keras.models import Model
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.losses import mean_squared_error, mean_absolute_error

    inputs = Input(shape=input_shape)
    conv_1 = Convolution2D(24, kernel_size=(5, 5), kernel_regularizer=l2(0.0005), strides=(2, 2), padding="same", activation="elu")(inputs)
    conv_2 = Convolution2D(36, kernel_size=(5, 5), kernel_regularizer=l2(0.0005), strides=(2, 2), padding="same", activation="elu")(conv_1)
    conv_3 = Convolution2D(48, kernel_size=(5, 5), kernel_regularizer=l2(0.0005), strides=(2, 2), padding="same", activation="elu")(conv_2)
    conv_4 = Convolution2D(64, kernel_size=(3, 3), kernel_regularizer=l2(0.0005), padding="same", activation="elu")(conv_3)
    conv_5 = Convolution2D(64, kernel_size=(3, 3), kernel_regularizer=l2(0.0005), padding="same", activation="elu")(conv_4)
    flatten = Flatten()(conv_5)
    dense_1 = Dense(1164, kernel_regularizer=l2(0.0005), activation="elu")(flatten)
    dense_2 = Dense(100, kernel_regularizer=l2(0.0005), activation="elu")(dense_1)
    dense_3 = Dense(50, kernel_regularizer=l2(0.0005), activation="elu")(dense_2)
    dense_4 = Dense(10, kernel_regularizer=l2(0.0005), activation="elu")(dense_3)
    out_dense = Dense(output_shape, activation=activation)(dense_4)

    model = Model(inputs=inputs, outputs=out_dense)
    optimizer = Adam(lr=3e-4)
    model.compile(loss=mean_absolute_error, optimizer=optimizer)

    return model

# Handy Testing

In [6]:
import json
import numpy as np
import os
import matplotlib.pyplot as plt
import skimage.io as io
import skimage
from skimage.transform import rescale, resize, downscale_local_mean

import glob
import random
from PIL import Image
import PIL
 

In [7]:
import glob

# val_data_loc="preprocessed/cleaned_all"
# filenames = glob.glob("preprocessed/cleaned_all/*")
val_data_loc="../cleaned_all/"
filenames = glob.glob("../cleaned_all/*")
print(filenames[1])
print(int(len(filenames)/2))
nr_of_datapoints = int(len(filenames)/2) #label and image files
print(nr_of_datapoints)


MAEs=[]
preds=[]
labels=[]


../cleaned_all/frame_0083594.npy
104215
104215


In [8]:
frames = []
commands = []
# for batch in range(53482,53482+nr_of_datapoints): # using the end of file. 32 batches of size batch of 32
for batch in range(1,nr_of_datapoints): # using the end of file. 32 batches of size batch of 32
    #frames=np.zeros((1,60,180,3))
    #commands = np.zeros((1,2))
    
#     frames.append(np.load("preprocessed/cleaned_all/frame_"+str(batch).zfill(7)+".npy"))
#     commands.append(np.load("preprocessed/cleaned_all/commands_"+str(batch).zfill(7)+".npy"))
#     frames.append(np.load(val_data_loc + "frame_n1_m1_"+str(batch).zfill(7)+".npy"))
#     commands.append(np.load(val_data_loc + "commands_n1_m1_"+str(batch).zfill(7)+".npy"))
    frames.append(np.load(val_data_loc + "frame_"+str(batch).zfill(7)+".npy"))
    commands.append(np.load(val_data_loc + "commands_"+str(batch).zfill(7)+".npy"))
    
    
    #print(commands)
    
    
    
    #MAEs.append(model.evaluate(frames,commands, batch_size=32))
    #pred = model.predict(frames)
    #preds.append(pred)
    #labels.append(commands)

In [9]:
#frames = frames.reshape(1,60,180,3)
frames = np.array(frames)
frames.shape

(104214, 60, 180, 3)

In [10]:
#commands = commands.reshape(1, 2)
commands = np.array(commands)
commands.shape

(104214, 2)

### Fitting with 10 Epochs

In [11]:
model = create_standalone_nvidia_cnn(activation='linear', input_shape=(60, 180, 3), output_shape=2)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 60, 180, 3)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 90, 24)        1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 45, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 23, 48)         43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 23, 64)         27712     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 23, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 11776)             0     

In [12]:
model.fit(frames, commands, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
1303/1303 [==============================] - 244s 187ms/step - loss: 2.6507 - val_loss: 0.9866
Epoch 2/10
1303/1303 [==============================] - 245s 188ms/step - loss: 0.8927 - val_loss: 0.6695
Epoch 3/10
1303/1303 [==============================] - 245s 188ms/step - loss: 0.6195 - val_loss: 0.5014
Epoch 4/10
1303/1303 [==============================] - 246s 189ms/step - loss: 0.4724 - val_loss: 0.4020
Epoch 5/10
1303/1303 [==============================] - 245s 188ms/step - loss: 0.3870 - val_loss: 0.3442
Epoch 6/10
1303/1303 [==============================] - 246s 189ms/step - loss: 0.3356 - val_loss: 0.3151
Epoch 7/10
1303/1303 [==============================] - 247s 190ms/step - loss: 0.3067 - val_loss: 0.2339
Epoch 8/10
1303/1303 [==============================] - 247s 189ms/step - loss: 0.1980 - val_loss: 0.1459
Epoch 9/10
1303/1303 [==============================] - 245s 188ms/step - loss: 0.1648 - val_loss: 0.1316
Epoch 10/10
1303/1303 [=======================

In [13]:
mem_frame = frames[10].reshape(1,60,180,3)
mem_frame.shape
mem_frame

new_mem_frame = mem_frame

if(new_mem_frame.all() == mem_frame.all()):
    print("sama")

sama


In [14]:
model.predict(mem_frame)

array([[0.07108895, 0.5243846 ]], dtype=float32)

In [15]:
from datetime import datetime

# Get current timestamp | source: https://www.programiz.com/python-programming/datetime/current-datetime
now = datetime.now().strftime("%Y%m%d_%H%M%S")
print(now)

#plot_model(model, to_file=model_path + model_file_prefix + model_file_suffix.format(model_number, 'png'), show_shapes=True)
model.save('../src/model_team_3_' + now + '_10epochs_ca1_ca2_ca3.h5')

20210117_012558


### Fitting with 12 Epochs

In [16]:
model2 = create_standalone_nvidia_cnn(activation='linear', input_shape=(60, 180, 3), output_shape=2)
model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 60, 180, 3)]      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 90, 24)        1824      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 45, 36)        21636     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 23, 48)         43248     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 23, 64)         27712     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 23, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 11776)             0   

In [17]:
model2.fit(frames, commands, batch_size=64, epochs=12, validation_split=0.2)

Epoch 1/12
1303/1303 [==============================] - 245s 188ms/step - loss: 1.5704 - val_loss: 0.7484
Epoch 2/12
1303/1303 [==============================] - 245s 188ms/step - loss: 0.6821 - val_loss: 0.4649
Epoch 3/12
1303/1303 [==============================] - 246s 189ms/step - loss: 0.4301 - val_loss: 0.3224
Epoch 4/12
1303/1303 [==============================] - 246s 189ms/step - loss: 0.3051 - val_loss: 0.2478
Epoch 5/12
1303/1303 [==============================] - 247s 189ms/step - loss: 0.2412 - val_loss: 0.1919
Epoch 6/12
1303/1303 [==============================] - 245s 188ms/step - loss: 0.2050 - val_loss: 0.1801
Epoch 7/12
1303/1303 [==============================] - 245s 188ms/step - loss: 0.1853 - val_loss: 0.1633
Epoch 8/12
1303/1303 [==============================] - 245s 188ms/step - loss: 0.1745 - val_loss: 0.1568
Epoch 9/12
1303/1303 [==============================] - 244s 188ms/step - loss: 0.1664 - val_loss: 0.1599
Epoch 10/12
1303/1303 [=======================

In [18]:
mem_frame = frames[10].reshape(1,60,180,3)
mem_frame.shape
mem_frame

new_mem_frame = mem_frame

if(new_mem_frame.all() == mem_frame.all()):
    print("sama")

sama


In [19]:
model2.predict(mem_frame)

array([[0.05913497, 0.4481774 ]], dtype=float32)

In [20]:
from datetime import datetime

# Get current timestamp | source: https://www.programiz.com/python-programming/datetime/current-datetime
now = datetime.now().strftime("%Y%m%d_%H%M%S")
print(now)

#plot_model(model, to_file=model_path + model_file_prefix + model_file_suffix.format(model_number, 'png'), show_shapes=True)
model2.save('../src/model_team_3_' + now + '_12epochs_ca1_ca2_ca3.h5')

20210117_021502
